In [85]:
import math
import numpy as np
from decimal import *
getcontext().prec = 10

### Dados de entrada

In [86]:
# valor das cargas aplicadas
f = [[32000,-16000]]


#ponto de aplicação das cargas
F = [0.500,0.160]

#posicionamento dos parafusos
p = [[0,0.120], [0.150, 0.120], [0.150,0], [0,0]]

#áreas dos parafusos
a = [[1],[1],[1],[1]]

tensao_cis = Decimal(170.95e6)
tensao_compressao = Decimal(214.72e6)
t=Decimal(10)

### Cálculo do Centroide

In [87]:
#Cálculo do centroide 
i = 0
b = []
c = []

for i in range(0,len(p)):

  x_barra = (p[i][0] * a[i][0]) / len(p) * a[i][0]
  y_barra = (p[i][1] * a[i][0]) / len(p) * a[i][0]

  x_barra = b.append(x_barra)
  y_barra = c.append(y_barra)

  x_barra = sum(b)
  y_barra = sum(c)

  i = i + 1

G = [x_barra,y_barra]

In [88]:
G

[0.075, 0.06]

### Distância entre o centróide e ponto de aplicação das cargas (GF)

In [89]:
#Distância entre o centroide e o ponto de aplicação das cargas (GF)
GF = [F[0]-G[0],F[1]-G[1]]

In [90]:
GF

[0.425, 0.1]

### Distância entre o parafuso e o centróide (ri)

In [91]:
i = 0
r = []

while i < len(p):  
  x = math.sqrt((p[i][0] - G[0])**2  + (p[i][1] - G[1])**2)
  r.append(x)

  i = i+1

In [92]:
r

[0.09604686356149274,
 0.09604686356149274,
 0.09604686356149274,
 0.09604686356149274]

### Momento resultante

In [93]:
def prod_vet(matriz_posicao, matriz_forca):
  i = 0
  a = []

  while i < len(matriz_posicao):
      prod = np.cross(matriz_posicao[i], matriz_forca[i])
      prod = a.append(prod)
      prod = sum(a)
      i = i + 1
  
  return prod

In [94]:
j = 0
MR = []
while j < len(f):
  y = prod_vet([GF], [f[j]])
  MR.append(y)
  j = j + 1

In [95]:
MR

[-10000.0]

In [96]:
MR = sum(MR)

In [97]:
MR

-10000.0

### Vetor GA (entre o centróide e ponto de aplicação da carga no parafuso)

In [98]:
#Vetor GA
i = 0
x = []

while i < len(p):
  GA = [p[i][0] - G[0],p[i][1]-G[1]]
  x.append(GA)
  i = i + 1

GA = x
print(GA)

[[-0.075, 0.06], [0.075, 0.06], [0.075, -0.06], [-0.075, -0.06]]


### Vetor Q (perpendicular ao GA)

In [99]:
# vetor Q
if MR > 0:
  i = 0
  while i < len(GA):
    GA[i][0],GA[i][1] = -GA[i][1],GA[i][0]  
    i = i + 1
else:
  i = 0
  while i < len(GA):
    GA[i][0],GA[i][1] = GA[i][1],-GA[i][0]
    i = i + 1
Q = GA
print(Q)

[[0.06, 0.075], [0.06, -0.075], [-0.06, -0.075], [-0.06, 0.075]]


### Vetor unitário de Q (u)
(saber a direção)


In [100]:
#cálculo do vetor unitário para saber a direção de Q
i = 0 
x = []
while i < len(Q):
  try:
    u = [Q[i][0]/math.sqrt( ((Q[i][0])**2)  + ((Q[i][1])**2) ),Q[i][1]/math.sqrt( ((Q[i][0])**2)  + ((Q[i][1])**2) )]
    x.append(u)
  except:
    u = [0.0,0.0]
    x.append(u)
  i = i + 1

u = x
print(u)  

[[0.6246950475544242, 0.7808688094430303], [0.6246950475544242, -0.7808688094430303], [-0.6246950475544242, -0.7808688094430303], [-0.6246950475544242, 0.7808688094430303]]


### Força resultante Q

Segundo a apostila, para achar o valor de Q, fazemos:
Q1 = r1 * Mg  / r1² + r2² + ... + rn²

In [101]:
r

[0.09604686356149274,
 0.09604686356149274,
 0.09604686356149274,
 0.09604686356149274]

In [102]:

#tratamento de dados para elevar a segunda potência todos os elementos da matriz r e depois somá-los.
i = 0
x = []
while i < len(r):
  a = r[i]**2
  x.append(a)
  i = i+1 

a = sum(x)

In [103]:
i = 0
q = []

while i < len(r):
  Q = r[i]*MR / a
  q.append(Q)
  i = i + 1

In [104]:
q

[-26028.96031476768,
 -26028.96031476768,
 -26028.96031476768,
 -26028.96031476768]

### Multiplicando a força Q pelo vetor unitário
(assim eu tenho um vetor Qi+Qj)

In [105]:
#multiplicando a força Q pelo vetor unitário
i = 0
x = []

while i < len(q):
  Q = [q[i]*u[i][0],q[i]*u[i][1]]
  x.append(Q)
  i = i + 1

Q = x
print(Q)

[[-16260.162601626014, -20325.20325203252], [-16260.162601626014, 20325.20325203252], [16260.162601626014, 20325.20325203252], [16260.162601626014, -20325.20325203252]]


### Encontrar as forças F do parafuso mais tensionado

In [106]:
i = 0
x = []

while i < len(p):
  F = [Q[i][0] + (f[0][0]/len(p)), Q[i][1] + (f[0][1]/len(p))]
  x.append(F)
  i = i + 1
x

[[-8260.162601626014, -24325.20325203252],
 [-8260.162601626014, 16325.20325203252],
 [24260.162601626012, 16325.20325203252],
 [24260.162601626012, -24325.20325203252]]

In [107]:
i = 0
y = []
while i < len(Q):
  F = [math.sqrt(x[i][0]**2 + x[i][1]**2)]
  y.append(F)
  i = i + 1

F = y
F

[[25689.410259054108],
 [18295.97079756836],
 [29241.54152361853],
 [34355.07244512849]]

In [108]:
Fi = max(F)

In [109]:
Fi

[34355.07244512849]

In [110]:
Fi = Decimal(Fi[0])

In [111]:
type(Fi)

decimal.Decimal

### Cálculo do diâmetro

In [112]:
d1 = math.sqrt((4*Fi)/(Decimal(3.14)*tensao_cis))

d2 = Fi/(t*tensao_compressao)

if d1>d2:
  print(d1)
else:
  print(d2)

0.016000223657811785
